In [2]:
import pandas as pd
from datetime import date, timedelta
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()

# SMA

In [3]:
def SMA(trend,slot,ndate):
    mv = 0
    sdate = date(2021,10,1)
    before = ndate - sdate
    before = int(before.total_seconds()/(60*60*24)) + 1

    if before >= slot:
        data = []
        for i in range(slot):
            temp_date = ndate - timedelta(i)
            temp_date = '_'.join(str(temp_date).split('-')) 
            temp_trend = pd.read_json('../assets/Result/newmm/'+str(temp_date)+'.json',encoding="utf8")
            temp_trend = temp_trend[temp_trend['LCS'] == trend]
            temp_trend.drop('total_match',axis='columns',inplace = True)

            if len(temp_trend) != 0:
                data.append(temp_trend.iloc[0]["match"])
            else:
                data.append(0)
        
        mv = round(sum(data)/slot, 2)
    return mv

In [ ]:
month = 12
day = 31

# for i in tqdm(range(1,5)):
#     df = pd.read_json(f"../assets/Result/newmm/2021_{month:02}_{i:02}.json", encoding="utf8")
#     df.rename(columns = {'match': 'frequency'}, inplace = True)
#     df["SMA"] = [0] * len(df)
#     df["dif"] = df["frequency"]
#     df.to_json(f"../assets/Result/MA/SMA/MINUS/2021_{month:02}_{i:02}.json", orient="records")
#     df.to_json(f"../assets/Result/MA/SMA/DIVIDE/2021_{month:02}_{i:02}.json", orient="records")

for i in tqdm(range(1,day+1)):
    df = pd.read_json(f"../assets/Result/newmm/2021_{month:02}_{i:02}.json", encoding="utf8")
    df.rename(columns = {'match': 'frequency'}, inplace = True)
    mv = []
    for lcs in df["LCS"]:
        avg = SMA(lcs, 5, date(2021,month,i))
        mv.append(avg)
    df["SMA"] = mv

    array1 = np.array(list(df["frequency"]))
    array2 = np.array(mv)

    subtracted_array = np.subtract(array1, array2)
    subtracted = list(subtracted_array)

    df["dif"] = subtracted
    df.sort_values(['dif'], ascending=False, inplace=True, ignore_index=True)
    df.to_json(f"../assets/Result/MA/SMA/MINUS/2021_{month:02}_{i:02}.json", orient="records")


    divided_array = np.divide(array1, array2)
    divided_array = np.around(divided_array, decimals=2)
    divided = list(divided_array)

    df["dif"] = divided
    df.sort_values(['dif'], ascending=False, inplace=True, ignore_index=True)
    df.to_json(f"../assets/Result/MA/SMA/DIVIDE/2021_{month:02}_{i:02}.json", orient="records")

# EMA

In [4]:
def EMA(trend,slot,ndate):
    mv = 0
    sdate = date(2021,10,slot)
    before = ndate - sdate
    before = int(before.total_seconds()/(60*60*24)) + 1

    if before > 0:
        for i in range(before):
            start = sdate + timedelta(i)
            # First Round Of EMA = Calculate SMA
            if i == 0:
                mv = SMA(trend,slot,start)
            # After that, Calculate with EMA formula
            else:
                temp_date = '_'.join(str(start).split('-'))
                temp_trend = pd.read_json('../assets/Result/newmm/'+str(temp_date)+'.json',encoding="utf8")
                temp_trend = temp_trend[temp_trend['LCS'] == trend]
                temp_trend.drop('total_match',axis='columns',inplace = True)

                if len(temp_trend) != 0:
                    v = temp_trend.iloc[0]["match"]
                    mv = (v - mv) * (2 / (1 + slot)) + mv
                else:
                    mv = (0 - mv) * (2 / (1 + slot)) + mv
    elif before == 0:
        mv = SMA(trend,slot,ndate)
    return round(mv, 2)

In [8]:
month = 12
day = 31

# for i in tqdm(range(1,5)):
#     df = pd.read_json(f"../assets/Result/newmm/2021_{month:02}_{i:02}.json", encoding="utf8")
#     df.rename(columns = {'match': 'frequency'}, inplace = True)
#     df["EMA"] = [0] * len(df)
#     df["dif"] = df["frequency"]
#     df.to_json(f"../assets/Result/MA/EMA/MINUS/2021_{month:02}_{i:02}.json", orient="records")
#     df.to_json(f"../assets/Result/MA/EMA/DIVIDE/2021_{month:02}_{i:02}.json", orient="records")

for i in tqdm(range(1,day+1)):
    df = pd.read_json(f"../assets/Result/newmm/2021_{month:02}_{i:02}.json", encoding="utf8")
    df.rename(columns = {'match': 'frequency'}, inplace = True)
    mv = []
    for lcs in df["LCS"]:
        avg = EMA(lcs, 5, date(2021,month,i))
        mv.append(avg)
    df["EMA"] = mv

    array1 = np.array(list(df["frequency"]))
    array2 = np.array(mv)

    subtracted_array = np.subtract(array1, array2)
    subtracted = list(subtracted_array)

    df["dif"] = subtracted
    df.sort_values(['dif'], ascending=False, inplace=True, ignore_index=True)
    df.to_json(f"../assets/Result/MA/EMA/MINUS/2021_{month:02}_{i:02}.json", orient="records")


    divided_array = np.divide(array1, array2)
    divided_array = np.around(divided_array, decimals=2)
    divided = list(divided_array)

    df["dif"] = divided
    df.sort_values(['dif'], ascending=False, inplace=True, ignore_index=True)
    df.to_json(f"../assets/Result/MA/EMA/DIVIDE/2021_{month:02}_{i:02}.json", orient="records")

100%|██████████| 31/31 [2:03:57<00:00, 239.93s/it]  


# Visualize

In [98]:
import pandas as pd

month, day = 12, 31
df = pd.read_json(f"../assets/Result/MA/EMA/DIVIDE/2021_{month:02}_{day:02}.json", encoding="utf8")
df.sort_values(['dif','frequency'], ascending=False, inplace=True, ignore_index=True)
df.head(20)

,LCS,frequency,total_match,EMA,dif
0,เพลงไทยสากลโดรฟิน,1,821,0.33,3.03
1,"3,1112,9571542,194,572",1,821,0.33,3.03
2,CoronaVacxboosterเอาไม่อยู่ไม่ต้องสงสัยCoronaV...,1,821,0.33,3.03
3,CoronaVacxboosterOmicronantibodyx1.4boost,1,821,0.33,3.03
4,มูหะพรรคประชาชาติ,1,821,0.33,3.03
5,ร่วมมือร่วมใจยื่นมือความสามัคคีเต็มเปี่ยมศักรา...,1,821,0.33,3.03
6,บับเบิลซีล,1,821,0.33,3.03
7,"IceParisBillkinPPKrit์,อิ้งค์รัน",1,821,0.33,3.03
8,จางหลินเผิง,1,821,0.33,3.03
9,กรมปศุสัตว์ไฟไหม้ช็อกโกแลตวิลล์ติดป้ายท้องเสีย...,1,821,0.33,3.03


In [85]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()

month, day = 12, 31

for i in tqdm(range(5,day+1)):
    df = pd.read_json(f"../assets/Result/MA/EMA/DIVIDE/2021_{month:02}_{i:02}.json", encoding="utf8")

    freq = list(df["frequency"])
    ma = list(df["EMA"])

    dif = [round((i / j),2) for i, j in zip(freq, ma)]

    df["dif"] = dif
    df.sort_values(['dif','frequency'], ascending=False, inplace=True, ignore_index=True)
    df.to_json(f"../assets/Result/MA/EMA/DIVIDE/2021_{month:02}_{i:02}.json", orient="records")
    # df.head(20)

100%|██████████| 27/27 [00:01<00:00, 20.55it/s]
